# Introduction

<p style='font-size:16px;'>Millennials are reaching the ages of getting married and having children, which prompts them to buy their first homes. However, there are so many factors to take into account when it comes to picking a suitable neighborhood, such as housing prices, pre-k to 12 schools, etc. <br> <br>Therefore, this project will use location data well as real estate data in New York City to recommend suitable neighborhoods for purchaing homes for people, especially couples with the intention to raise their kids in NYC. In addition, these neighborhoods can also be of interest for real estate company who would like to develop new projects. </p>

# Data

<p style='font-size:16px;'> Location data from Foursquare API as well as real estate data from StreetEasy will be utilised for the analysis. <br><br> Real estate data, namely median sales price as well as inventory of houses in each neighborhood in New York City will be combined with location data that home purchasers are interested in, including categories such as school, Doctor's Office, Outdoors & Recreation, etc. 
</p>

<p style='font-size:16px;'>Coordinates of NYC neighborhood will be obtained from IBM dataset, and up to 100 venues within 1km radius from each neighborhood coordinate in the following categories will be selected:

<ul style='font-size:16px;'>
    <li> Food & Drink Shop </li>
    <li> School </li>
    <li> Doctor's Office </li>
    <li> Outdoors & Recreation </li>
    <li> Residence</li>
</ul>
</p>

<p style='font-size:16px;'> Average total inventory and median price of houses for sale in first quarter of 2020 in each neighborhood of NYC will be obtained from StreeEasy as csv files, which will be employed as features in addition to the data of various venues in each neighborhood. <br> <br> These data will be merged into one data frame. After normalization, all the features will be fed to K-Means clustering algorithm for neighborhood segmentation. Afterwards, neighborhoods will be divided into 5 differnt groups to facilitate the process of picking a suitable neighborhood to purchase a home. </p>

# Methodology

## Import Libraries

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.') 

In [1]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


# Results

# Discussion

# Conclusion